In [ ]:
#importing necessary libraries
import pandas as pd
import string
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#loading the dataset after converting tsv into csv
train_data = pd.read_csv("/content/drive/MyDrive/olid-training.csv")
train_data

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13188,94475,@USER every antifa member is just several midg...,NOT,NaN,NaN
13189,34150,@USER Berkeley Antifa do not agree with you URL,NOT,NaN,NaN
13190,83400,@USER You are a great model for inspiration,NOT,NaN,NaN
13191,84081,"@USER Chris, Chris, Chris.... Are you forgetti...",OFF,UNT,NaN


In [ ]:
'''from sklearn.utils import resample
import pandas as pd

# Separate the majority and minority classes
majority_class = train_data[train_data['subtask_a'] == 'NOT']
minority_class = train_data[train_data['subtask_a'] == 'OFF']

# Checking counts of majority and minority classes
print("Original Counts - Majority:", len(majority_class), "Minority:", len(minority_class))

# Oversample the minority class to match the number of majority class instances
oversampled_minority = resample(minority_class,
                                replace=True,  # Sample with replacement
                                n_samples=len(majority_class),  # Match number of majority class instances
                                random_state=42)  # Reproducible results

# Combine oversampled minority class with majority class
balanced_data = pd.concat([majority_class, oversampled_minority])


print("Balanced Data Shape:", balanced_data.shape)'''

'from sklearn.utils import resample\nimport pandas as pd\n\n# Separate the majority and minority classes\nmajority_class = train_data[train_data[\'subtask_a\'] == \'NOT\']\nminority_class = train_data[train_data[\'subtask_a\'] == \'OFF\']\n\n# Checking counts of majority and minority classes\nprint("Original Counts - Majority:", len(majority_class), "Minority:", len(minority_class))\n\n# Oversample the minority class to match the number of majority class instances\noversampled_minority = resample(minority_class,\n                                replace=True,  # Sample with replacement\n                                n_samples=len(majority_class),  # Match number of majority class instances\n                                random_state=42)  # Reproducible results\n\n# Combine oversampled minority class with majority class\nbalanced_data = pd.concat([majority_class, oversampled_minority])\n\n\nprint("Balanced Data Shape:", balanced_data.shape)'

In [ ]:
#displaying the tweets
train_tweets = train_data[["tweet"]]
print(train_tweets)

                                                   tweet
0      @USER She should ask a few native Americans wh...
1      @USER @USER Go home you’re drunk!!! @USER #MAG...
2      Amazon is investigating Chinese employees who ...
3      @USER Someone should'veTaken" this piece of sh...
4      @USER @USER Obama wanted liberals &amp; illega...
...                                                  ...
13188  @USER every antifa member is just several midg...
13189    @USER Berkeley Antifa do not agree with you URL
13190        @USER You are a great model for inspiration
13191  @USER Chris, Chris, Chris.... Are you forgetti...
13192  @USER Advocate for gun control while breaking ...

[13193 rows x 1 columns]


In [ ]:
 #checking for duplicate tweets
 train_data["tweet"].duplicated().sum()

33

In [ ]:
#value counts of subtask a after balancing
train_data[['subtask_a']].value_counts()

subtask_a
NOT          8804
OFF          4389
dtype: int64

In [ ]:
train_task_a_labels= train_data[['subtask_a']] #Extract subtsak_a labels
train_task_b_labels= train_data[['subtask_b']] #Extract subtsak_b labels
train_task_c_labels= train_data[['subtask_c']] #Extract subtsak_c labels

train_task_a_labels.columns.values[0] = 'class_a' #Rename class attribute
train_task_b_labels.columns.values[0] = 'class_b' #Rename class attribute
train_task_c_labels.columns.values[0] = 'class_c' #Rename class attribute

print(train_data)

          id                                              tweet subtask_a  \
0      86426  @USER She should ask a few native Americans wh...       OFF   
1      90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF   
2      16820  Amazon is investigating Chinese employees who ...       NOT   
3      62688  @USER Someone should'veTaken" this piece of sh...       OFF   
4      43605  @USER @USER Obama wanted liberals &amp; illega...       NOT   
...      ...                                                ...       ...   
13188  94475  @USER every antifa member is just several midg...       NOT   
13189  34150    @USER Berkeley Antifa do not agree with you URL       NOT   
13190  83400        @USER You are a great model for inspiration       NOT   
13191  84081  @USER Chris, Chris, Chris.... Are you forgetti...       OFF   
13192  11244  @USER Advocate for gun control while breaking ...       NOT   

      subtask_b subtask_c  
0           UNT       NaN  
1           TIN    

In [ ]:


def clean_tweets(df):
    punctuations = string.punctuation
    stop_words = set(stopwords.words('english'))

    # Remove mentions, URLs, and special characters
    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') \
                                    .str.replace('URL', '') \
                                    .str.replace('&amp', 'and') \
                                    .str.replace('&lt','') \
                                    .str.replace('&gt','') \
                                    .str.replace('\d+','') \
                                    .str.lower()

    # Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')

    # Remove emojis
    df.loc[:, 'tweet'] = df.tweet.apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))

    # Remove stopwords
    df.loc[:, 'tweet'] = df.tweet.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    # Trim leading and trailing whitespaces
    df.loc[:, 'tweet'] = df.tweet.str.strip()

    return df

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
clean_tweets(train_tweets)

<ipython-input-10-0e83b4979b94>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('\d+','') \
<ipython-input-10-0e83b4979b94>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') \
<ipython-input-10-0e83b4979b94>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')
<ipython-input-10-0e83b4979b94>:16: SettingWithCopyWarning: 
A value is tryi

,tweet
0,ask native americans take
1,go home youre drunk maga trump
2,amazon investigating chinese employees selling...
3,someone shouldvetaken piece shit volcano
4,obama wanted liberals illegals move red states
...,...
13188,every antifa member several midget gypsies sta...
13189,berkeley antifa agree
13190,great model inspiration
13191,chris chris chris forgetting mantra every sing...


In [ ]:
train_task_a_data = train_tweets.join(train_task_a_labels)

train_task_b_data = train_tweets.join(train_task_b_labels)
train_task_b_data = train_task_b_data.dropna() #Drop records with missing values

train_task_c_data = train_tweets.join(train_task_c_labels)
train_task_c_data = train_task_c_data.dropna() #Drop records with missing values

In [ ]:
from nltk.tokenize import word_tokenize

def tokenize_tweets(tweets):

    tokenized_tweets = [word_tokenize(tweet) for tweet in tweets]
    return tokenized_tweets


#  train_tweets['tweet'] contains the list of tweets
tweets = train_tweets['tweet']
tokenized_tweets = tokenize_tweets(tweets)

# Printing the first few tokenized tweets for verification
for i in range(5):  # Print the first 5 tokenized tweets
    print(f"Tweet {i + 1}: {tokenized_tweets[i]}")


Tweet 1: ['ask', 'native', 'americans', 'take']
Tweet 2: ['go', 'home', 'youre', 'drunk', 'maga', 'trump']
Tweet 3: ['amazon', 'investigating', 'chinese', 'employees', 'selling', 'internal', 'data', 'thirdparty', 'sellers', 'looking', 'edge', 'competitive', 'marketplace', 'amazon', 'maga', 'kag', 'china', 'tcot']
Tweet 4: ['someone', 'shouldvetaken', 'piece', 'shit', 'volcano']
Tweet 5: ['obama', 'wanted', 'liberals', 'illegals', 'move', 'red', 'states']


In [ ]:
#Lemmatization preserves the semantic meaning of words better than stemming and is less likely to introduce non-standard words.
# lemmatization also considers different POS tags
#lemm using nltk

from nltk.stem import WordNetLemmatizer

def lemmatize_tweets(tokenized_tweets):

    lemmatizer = WordNetLemmatizer()
    lemmatized_tweets = []
    for tweet_tokens in tokenized_tweets:
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tweet_tokens]
        lemmatized_tweets.append(lemmatized_tokens)
    return lemmatized_tweets

# Assuming tokenized_tweets contains the list of tokenized tweets
lemmatized_tweets = lemmatize_tweets(tokenized_tweets)

# Printing the first few lemmatized tweets for verification
for i in range(5):  # Print the first 5 lemmatized tweets
    print(f"Lemmatized Tweet {i + 1}: {lemmatized_tweets[i]}")
print(len(lemmatized_tweets))

Lemmatized Tweet 1: ['ask', 'native', 'american', 'take']
Lemmatized Tweet 2: ['go', 'home', 'youre', 'drunk', 'maga', 'trump']
Lemmatized Tweet 3: ['amazon', 'investigating', 'chinese', 'employee', 'selling', 'internal', 'data', 'thirdparty', 'seller', 'looking', 'edge', 'competitive', 'marketplace', 'amazon', 'maga', 'kag', 'china', 'tcot']
Lemmatized Tweet 4: ['someone', 'shouldvetaken', 'piece', 'shit', 'volcano']
Lemmatized Tweet 5: ['obama', 'wanted', 'liberal', 'illegals', 'move', 'red', 'state']
13193


In [ ]:
train_tweets["tweet"]

0                                ask native americans take
1                           go home youre drunk maga trump
2        amazon investigating chinese employees selling...
3                 someone shouldvetaken piece shit volcano
4           obama wanted liberals illegals move red states
                               ...                        
13188    every antifa member several midget gypsies sta...
13189                                berkeley antifa agree
13190                              great model inspiration
13191    chris chris chris forgetting mantra every sing...
13192    advocate gun control breaking current gun laws...
Name: tweet, Length: 13193, dtype: object

In [ ]:
'''#word cloud after pre-processing
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Sample lemmatized_tweets data (replace it with your actual lemmatized tweets)
lemmatized_tweets =[ train_tweets['tweet']]

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Combine all lemmatized tokens into a single text corpus
text_corpus = ' '.join([' '.join([lemmatizer.lemmatize(token) for token in tweet_tokens]) for tweet_tokens in lemmatized_tweets])

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_corpus)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
'''

"#word cloud after pre-processing\nfrom nltk.stem import WordNetLemmatizer\nfrom wordcloud import WordCloud\nimport matplotlib.pyplot as plt\n\n# Sample lemmatized_tweets data (replace it with your actual lemmatized tweets)\nlemmatized_tweets =[ train_tweets['tweet']]\n\n# Initialize the WordNetLemmatizer\nlemmatizer = WordNetLemmatizer()\n\n# Combine all lemmatized tokens into a single text corpus\ntext_corpus = ' '.join([' '.join([lemmatizer.lemmatize(token) for token in tweet_tokens]) for tweet_tokens in lemmatized_tweets])\n\n# Generate the word cloud\nwordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_corpus)\n\n# Display the word cloud using matplotlib\nplt.figure(figsize=(10, 6))\nplt.imshow(wordcloud, interpolation='bilinear')\nplt.axis('off')\nplt.show()\n"

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_vectorize_tweets(tweets, max_features=5000, ngram_range=(1, 3), stop_words='english'):

    # Initialize the TfidfVectorizer with desired parameters
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features, stop_words=stop_words, ngram_range=ngram_range)

    # Fit the vectorizer to the tweets and transform them into TF-IDF vectors
    tfidf_matrix = tfidf_vectorizer.fit_transform(tweets)

    # Get the feature names (words) from the vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()

    return tfidf_matrix, feature_names

# Example usage:
# Assuming lemmatized_tweet_strings contains the list of lemmatized tweet strings
lemmatized_tweet_strings = [' '.join(tokens) for tokens in lemmatized_tweets]
tfidf_matrix, feature_names = tfidf_vectorize_tweets(lemmatized_tweet_strings)

# Print the TF-IDF matrix and feature names
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())
print("\nFeature Names:")
print(feature_names)
print(tfidf_matrix.shape)


TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Feature Names:
['aaron' 'ab' 'abiding' ... 'zero evidence' 'zombie' 'zone']
(13193, 5000)


In [ ]:
'''vectorization using tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert the lemmatized tweets back to strings
lemmatized_tweet_strings = [' '.join(tokens) for tokens in lemmatized_tweets]

# Initializing the TfidfVectorizer with desired parameters
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 3))

# Fit the vectorizer to the lemmatized tweets and transform them into TF-IDF vectors
tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_tweet_strings)

# Get the feature names (words) from the vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Print the TF-IDF matrix and feature names
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())
print("\nFeature Names:")
print(feature_names)

print(tfidf_matrix.shape)'''

'vectorization using tfidf\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\n# Convert the lemmatized tweets back to strings\nlemmatized_tweet_strings = [\' \'.join(tokens) for tokens in lemmatized_tweets]\n\n# Initializing the TfidfVectorizer with desired parameters\ntfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=\'english\', ngram_range=(1, 3))\n\n# Fit the vectorizer to the lemmatized tweets and transform them into TF-IDF vectors\ntfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_tweet_strings)\n\n# Get the feature names (words) from the vectorizer\nfeature_names = tfidf_vectorizer.get_feature_names_out()\n\n# Print the TF-IDF matrix and feature names\nprint("TF-IDF Matrix:")\nprint(tfidf_matrix.toarray())\nprint("\nFeature Names:")\nprint(feature_names)\n\nprint(tfidf_matrix.shape)'

In [ ]:
#undersampling using Resample library
combined_data = pd.concat([pd.DataFrame(tfidf_matrix.toarray()), train_data['subtask_a']], axis=1)

# Separate majority and minority classes
majority_class = combined_data[combined_data['subtask_a'] == 'NOT']
minority_class = combined_data[combined_data['subtask_a'] == 'OFF']

# Upsample minority class
majority_downsampled = resample(majority_class,
                               replace=False,  # Sample with replacement
                               n_samples=len(minority_class),  # Match number in majority class
                               random_state=42)  # Reproducible results

# Combine majority class with upsampled minority class
downsampled_data = pd.concat([majority_downsampled, minority_class])

# Separate features (TF-IDF matrix) and labels
X_downsampled = downsampled_data.drop('subtask_a', axis=1)
y_downsampled = downsampled_data['subtask_a']
print(downsampled_data['subtask_a'].shape)

(8778,)


In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
import mlflow

In [ ]:
with mlflow.start_run():
    # Add parameters
    mlflow.log_param("C", 1)
    mlflow.log_param("penalty", 'l2')
    mlflow.log_param("random_state", 42)
#Split the upsampled data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_downsampled, y_downsampled, test_size=0.2, random_state=42)

# Initialize the logistic regression model
logistic_model = LogisticRegression(C = 1, penalty = 'l2',random_state=42)

# Fit the model to the training data
logistic_model.fit(X_train, y_train)

# Predict labels for the testing data
lr_pred = logistic_model.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, lr_pred)
print("Accuracy:", accuracy)

mlflow.log_metric("accuracy", accuracy)

#printing classification report
print(classification_report(y_val, lr_pred, output_dict=True))
for class_label, metrics in classification_rep.items():
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(f"{class_label}_{metric_name}", metric_value)

In [ ]:

'''
# Apply SMOTE to oversample the majority class
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = smote.fit_resample(tfidf_matrix, train_data['subtask_a'])

# Split the resampled TF-IDF matrix and labels into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize and fit the logistic regression model
logistic_model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
logistic_model.fit(X_train, y_train)

# Predicting labels for the validation set
y_pred_val = logistic_model.predict(X_val)

# Calculate accuracy score for validation set
accuracy_val = accuracy_score(y_val, y_pred_val)
print("Validation Accuracy:", accuracy_val)

# Predicting labels for the training set
y_pred_train = logistic_model.predict(X_train)

# Calculate accuracy score for training set
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", accuracy_train)'''

'\n# Apply SMOTE to oversample the majority class\nsmote = SMOTE(sampling_strategy=\'minority\', random_state=42)\nX_resampled, y_resampled = smote.fit_resample(tfidf_matrix, train_data[\'subtask_a\'])\n\n# Split the resampled TF-IDF matrix and labels into train and validation sets\nX_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)\n\n# Initialize and fit the logistic regression model\nlogistic_model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed\nlogistic_model.fit(X_train, y_train)\n\n# Predicting labels for the validation set\ny_pred_val = logistic_model.predict(X_val)\n\n# Calculate accuracy score for validation set\naccuracy_val = accuracy_score(y_val, y_pred_val)\nprint("Validation Accuracy:", accuracy_val)\n\n# Predicting labels for the training set\ny_pred_train = logistic_model.predict(X_train)\n\n# Calculate accuracy score for training set\naccuracy_train = accuracy_score(y_train, y_pred_train)\npri

In [ ]:
#lr model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming you have labels associated with your lemmatized tweets
# Replace 'labels' with the actual labels from your dataset

# Split the TF-IDF matrix and labels into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(tfidf_matrix, train_data['subtask_a'], test_size=0.2, random_state=42)

# Initialize and fit the logistic regression model
logistic_model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
logistic_model.fit(X_train, y_train)

# Predicting labels for the validation set
lr_pred = logistic_model.predict(X_val)

# accuracy score for validation set
accuracy = accuracy_score(y_val, lr_pred)
print("training Accuracy:", accuracy)

# Predicting labels for the training set
y_pred_tr = logistic_model.predict(X_train)

# Calculate accuracy score
accuracy = accuracy_score(y_train, y_pred_tr)
print("testing Accuracy:", accuracy)

training Accuracy: 0.7491474043198181
testing Accuracy: 0.8082243699071442


In [ ]:
#rf model


# Split the TF-IDF matrix and labels into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_downsampled, y_downsampled, test_size=0.2, random_state=42)

# Initialize and fit the Random Forest classifier
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)

# Predict labels for the validation set
rf_pred = random_forest_model.predict(X_val)

# Calculated accuracy score
accuracy = accuracy_score(y_val, rf_pred)
print("val_Accuracy:", accuracy)

# Predict labels for the training set
y_pred_train = random_forest_model.predict(X_train)

# Calculate accuracy score
accuracy_train = accuracy_score(y_train, y_pred_train)
print("train_Accuracy:", accuracy_train)

val_Accuracy: 0.6964692482915718
train_Accuracy: 0.9891768726858445


In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Initialize and fit the Random Forest classifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

dt_pred = dt_model.predict(X_val)
acc = accuracy_score(y_val, dt_pred)
print(acc)

0.6503416856492027


In [ ]:
# Convert predictions to binary (0 or 1) if they are not already
rf_pred = random_forest_model.predict_proba(X_val)[:, 1]  # Assuming you want the probability of the positive class
lr_pred = logistic_model.predict_proba(X_val)[:, 1]
dt_pred = dt_model.predict_proba(X_val)[:, 1]

# Convert probabilities to binary predictions using a threshold of 0.5
rf_pred_binary = (rf_pred >= 0.5).astype(int)
lr_pred_binary = (lr_pred >= 0.5).astype(int)
dt_pred_binary = (dt_pred >= 0.5).astype(int)

# Combine predictions using simple majority voting
ensemble_predictions = (rf_pred_binary + lr_pred_binary + dt_pred_binary) >= 2

# Calculate Accuracy
ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)
print("Ensemble Accuracy:", ensemble_accuracy)


In [ ]:
# Step 5: Combine Predictions
# Voting Ensemble (Simple Majority Voting)
ensemble_predictions = (rf_pred + lr_pred + dt_pred) >= 2
ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)
print("Ensemble Accuracy:", ensemble_accuracy)

In [ ]:
e'''from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(tfidf_matrix, train_data['subtask_a'], test_size=0.2, random_state=42)

# Apply SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Initialize Random Forest classifier
random_forest_model = RandomForestClassifier(random_state=42)

# Define hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=random_forest_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)

# Get the best model from grid search
best_random_forest_model = grid_search.best_estimator_

# Evaluate the model on the validation set
y_pred_val = best_random_forest_model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print("Validation Accuracy:", accuracy_val)

# Predict labels for the training set
y_pred_train = best_random_forest_model.predict(X_resampled)
accuracy_train = accuracy_score(y_resampled, y_pred_train)
print("Training Accuracy:", accuracy_train)
'''

In [ ]:

'''Support Vector Classifier
# Split the data into training and validation sets
X_train_svm, X_val_svm, y_train_svm, y_val_svm = train_test_split(tfidf_matrix, train_data['subtask_a'], test_size=0.2, random_state=42)

# Apply SMOTE to oversample the minority class
smote_svm = SMOTE(sampling_strategy='minority', random_state=42)
X_resampled_svm, y_resampled_svm = smote_svm.fit_resample(X_train_svm, y_train_svm)

# Initialize SVM classifier
svm_model = SVC(random_state=42)

# Define hyperparameters to tune
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_resampled_svm, y_resampled_svm)

# Get the best model from grid search
best_svm_model = grid_search.best_estimator_

# Evaluate the model on the validation set
y_pred_val_svm = best_svm_model.predict(X_val_svm)
accuracy_val_svm = accuracy_score(y_val_svm, y_pred_val_svm)
print("Validation Accuracy:", accuracy_val_svm)

# Predict labels for the training set
y_pred_train_svm = best_svm_model.predict(X_resampled_svm)
accuracy_train_svm = accuracy_score(y_resampled_svm, y_pred_train_svm)
print("Training Accuracy:", accuracy_train_svm)'''


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Split the upsampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_upsampled, y_upsampled, test_size=0.2, random_state=42)

# Define the maximum sequence length
max_len = X_train.shape[1]  # Assuming X_train is a matrix of shape (num_samples, max_features)

# Convert string labels to numerical format
y_train = y_train.map({'OFF': 1, 'NOT': 0})
y_test = y_test.map({'OFF': 1, 'NOT': 0})

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
print(model.summary())

# Train the model
history = model.fit(X_train.values, y_train.values, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=1)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Flatten
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
# Convert TF-IDF matrix to array
X_features = tfidf_matrix.toarray()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_features, y_downsampled, test_size=0.2, random_state=42)

# Reshape the data for CNN input (3D tensor required)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

TESTING

In [ ]:
#Read tweets from test sets
test_tweet_a=pd.read_csv("/content/drive/MyDrive/testset-levela.tsv", delimiter='\t', encoding='utf-8')

#Read tweet labels
test_label_a=pd.read_csv('/content/drive/MyDrive/labels-levela.csv', names=['id', 'class_a'])

#Merge tweets with labels by id
test_tweet_a = test_tweet_a.merge(test_label_a, on='id')

print(test_tweet_a.head)

In [ ]:
# Clean the tweets in the testing dataset
clean_tweets(test_tweet_a)

# Tokenize the cleaned tweets
tokenized_test_tweets = tokenize_tweets(test_tweet_a['tweet'])

# Lemmatize the tokenized tweets (if needed)
lemmatized_test_tweets = lemmatize_tweets(tokenized_test_tweets)

# Convert the lemmatized tweets back to strings
lemmatized_test_tweet_strings = [' '.join(tokens) for tokens in lemmatized_test_tweets]

# Vectorize the lemmatized tweet strings using TF-IDF
test_tfidf_matrix, _ = tfidf_vectorize_tweets(lemmatized_test_tweet_strings)

# Predict labels for the testing set
y_pred_test = random_forest_model.predict(test_tfidf_matrix)

# Print the predicted labels
print("Predicted Labels:", y_pred_test)


In [ ]:
y_true_test = test_tweet_a['class_a']
accuracy_test = accuracy_score(y_true_test, y_pred_test)
print("Test Accuracy:", accuracy_test)

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True